# Elasticity equation

In this tutorial we present how to solve the elasticity equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) and [PorePy](https://github.com/pmgbergen/porepy).  The unknown is the displacement $u$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force.

We will use the Multi-Point Stress Approximation (MPSA) to discretise the problem.

## Exercise 3: cantilever beam problem

A cantilever beam problem is when the body is fixed on one of the lateral boundary and subject to a body force.
For this test case we set $\Omega = [0, 2] \times [0, 1]$, the body force equal to $b = [0, -0.01]^\top$ and the following boundary conditions:
$$ 
u = 0 \text{ on } \partial_{left} \Omega 
\qquad 
\nu \cdot \sigma = 0 \text{ on } \partial \Omega \setminus \partial_{left} \Omega 
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [9]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [10]:
mesh_size = 0.025
dim = 2

bd_pts = np.array(
    [
        [0, 2, 2, 0],
        [0, 0, 1, 1],
    ]
)
sd = pg.grid_from_boundary_pts(bd_pts, mesh_size, as_mdg=False)
sd.compute_geometry()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [11]:
key = "elasticity"

lambda_ = 1
mu = 0.5

fun = -1e-2

Let us now put all the data together.

In [12]:
# Create stiffness matrix
lambda_ = lambda_ * np.ones(sd.num_cells)
mu = mu * np.ones(sd.num_cells) / 2
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["neu"] * num_b_faces)

left = np.isclose(sd.face_centers[0, b_faces], 0)
labels[left] = "dir"
bound = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_values = np.zeros((sd.dim, sd.num_faces))

right = np.isclose(sd.face_centers[0, :], 2)
bc_values[1, right] = fun * sd.face_areas[right]
bc_values = bc_values.ravel("F")

# No source term
source = np.zeros(sd.num_cells * sd.dim)

# collect all data
data = {
    pp.PARAMETERS: {
        key: {
            "fourth_order_tensor": C,
            "bc_values": bc_values,
            "bc": bound,
            "source": source,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key: {}},
}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [13]:
# discretize and solve the system
mpsa = pp.Mpsa(key)
mpsa.discretize(sd, data)

A, b = mpsa.assemble_matrix_rhs(sd, data)
u = sps.linalg.spsolve(A, b)

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [14]:
# reshape the displacement for the export
u = np.reshape(u, (sd.dim, -1), order="F")
u_3d = np.vstack((u, np.zeros(sd.num_cells)))

save = pp.Exporter(sd, "sol", folder_name="ex3")
save.write_vtu([("u", u_3d)])

In [15]:
# Consistency check
assert np.isclose(np.linalg.norm(u), 20.655724714254458)